In [19]:
import pandas as pd
import numpy as np
import os
from env import get_db_url
import wrangle as w

## 1.) Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.

In [20]:
# gets the requested dataframe from mysql zillow
def get_zillow_2017():
    filename = "zillow_2017.csv"
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
    else:
        query = """
        SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
        FROM properties_2017
        JOIN propertylandusetype
        USING (propertylandusetypeid)
        WHERE propertylandusetypeid like '261';"""
        connection = get_db_url("zillow")
        df = pd.read_sql(query, connection)
        df.to_csv(filename, index=False)
    return df

In [21]:
df = get_zillow_2017()

In [22]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


## 2.) Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.

In [23]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [24]:
df.shape

(2152863, 7)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
dtypes: float64(7)
memory usage: 115.0 MB


In [11]:
# why so many null values

df.isnull().sum()

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
dtype: int64

In [26]:
# we have a signigicant amount of missing values in a few of these fields,
# but is it a huge problem considering we have literally millions of rows?
df.isna().sum() / df.shape[0] * 100

bedroomcnt                      0.000511
bathroomcnt                     0.000511
calculatedfinishedsquarefeet    0.394080
taxvaluedollarcnt               0.022900
yearbuilt                       0.433702
taxamount                       0.206330
fips                            0.000000
dtype: float64

In [27]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
bedroomcnt,2152852.0,3.287196,0.954754,0.00,3.00,3.00,4.00,25.00
bathroomcnt,2152852.0,2.230688,0.999280,0.00,2.00,2.00,3.00,32.00
calculatedfinishedsquarefeet,2144379.0,1862.855178,1222.125124,1.00,1257.00,1623.00,2208.00,952576.00
taxvaluedollarcnt,2152370.0,461896.237963,699676.049600,1.00,188170.25,327671.00,534527.00,98428909.00
yearbuilt,2143526.0,1960.949681,22.162196,1801.00,1949.00,1958.00,1976.00,2016.00
taxamount,2148421.0,5634.865978,8178.910249,1.85,2534.98,4108.95,6414.32,1337755.86
fips,2152863.0,6048.377335,20.433292,6037.00,6037.00,6037.00,6059.00,6111.00


In [14]:
# renamed columns for personal pref
df.rename(columns={df.columns[0]: 'bedrooms', df.columns[1]: 'bathrooms', df.columns[2]: 'finished_sqft', df.columns[3]: 'tax_value'}, inplace=True)


In [15]:
df.head()

,bedrooms,bathrooms,finished_sqft,tax_value,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [29]:
df = df.dropna()

In [30]:
df.info(show_counts= True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2140235 non-null  float64
 1   bathroomcnt                   2140235 non-null  float64
 2   calculatedfinishedsquarefeet  2140235 non-null  float64
 3   taxvaluedollarcnt             2140235 non-null  float64
 4   yearbuilt                     2140235 non-null  float64
 5   taxamount                     2140235 non-null  float64
 6   fips                          2140235 non-null  float64
dtypes: float64(7)
memory usage: 130.6 MB


In [31]:
# lets check out the float values in the same manner as we did earlier today
# but iterated through each column:
# best way to build a loop:
# do the thing once:

(df['fips'] != df['fips'].astype(int)).sum()
# see what you can change the thing (column name)
# make the thing you can change an iterable

cast_to_int = []

for col in df.columns:
    sum_of_discord = (df[col] != df[col].astype(int)).sum()
    if sum_of_discord > 0:
        print(f'{col} has data that is probably not interpretable as an int! It has {sum_of_discord / df.shape[0] * 100} percent mismatched values!')
    else:
        print(f'{col} is good to go for int casting!')
        cast_to_int.append(col)
    print('============')

bedroomcnt is good to go for int casting!
bathroomcnt has data that is probably not interpretable as an int! It has 10.748072057507702 percent mismatched values!
calculatedfinishedsquarefeet is good to go for int casting!
taxvaluedollarcnt is good to go for int casting!
yearbuilt is good to go for int casting!
taxamount has data that is probably not interpretable as an int! It has 98.66299728768102 percent mismatched values!
fips is good to go for int casting!


In [32]:
cast_to_int

['bedroomcnt',
 'calculatedfinishedsquarefeet',
 'taxvaluedollarcnt',
 'yearbuilt',
 'fips']

In [33]:
for col in cast_to_int:
    df.loc[:,col] = df[col].astype(int)

/var/folders/jd/t05l8rl14_l105k8kn2tdng40000gn/T/ipykernel_87871/1764613563.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,col] = df[col].astype(int)


In [34]:
# column names? we can probably clean these up a bit:
df.columns
df = df.rename(columns={
    'bedroomcnt': 'bedrooms',
    'bathroomcnt': 'bathrooms',
    'calculatedfinishedsquarefeet': 'sqft',
    'taxvaluedollarcnt': 'taxvalue',
    'fips': 'county'
})
# fips?

In [37]:
df.county.value_counts()

6037    1425207
6059     552057
6111     162971
Name: county, dtype: int64

In [38]:
county_maps = {6037: 'LA',
6059: 'Orange',
6111: 'Ventura'
}

In [39]:
df['county'] = df.county.map(county_maps)

## 3.) Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values in your wrangle.py file. Name your final function wrangle_zillow.

In [43]:
# gets the requested dataframe from mysql zillow
def get_zillow_2017():
    filename = "zillow_2017.csv"
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
    else:
        query = """
        SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
        FROM properties_2017
        JOIN propertylandusetype
        USING (propertylandusetypeid)
        WHERE propertylandusetypeid like '261';"""
        connection = get_db_url("zillow")
        df = pd.read_sql(query, connection)
        df.to_csv(filename, index=False)
    return df


In [44]:
def prep_zillow(df):
    '''
    This function takes in a dataframe
    renames the columns and drops nulls values
    Additionally it changes datatypes for appropriate columns
    and renames fips to actual county names.
    Then returns a cleaned dataframe
    '''
    df = df.rename(columns = {'bedroomcnt':'bedrooms',
                     'bathroomcnt':'bathrooms',
                     'calculatedfinishedsquarefeet':'area',
                     'taxvaluedollarcnt':'taxvalue',
                     'fips':'county'})
    
    df = df.dropna()
    
    make_ints = ['bedrooms','area','taxvalue','yearbuilt']

    for col in make_ints:
        df[col] = df[col].astype(int)
        
    df.county = df.county.map({6037:'LA',6059:'Orange',6111:'Ventura'})
    
    return df

In [46]:
df = prep_zillow(get_zillow_2017())

In [47]:
df.head()

,bedrooms,bathrooms,area,taxvalue,yearbuilt,taxamount,county
4,4,2.0,3633,296425,2005,6941.39,LA
6,3,4.0,1620,847770,2011,10244.94,LA
7,3,2.0,2077,646760,1926,7924.68,LA
11,0,0.0,1200,5328,1972,91.60,LA
14,0,0.0,171,6920,1973,255.17,LA
